# Reliability anaysis


## Packages and Helpers

In [1]:
# packages
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from utils import load_dataframes
from scipy.stats import pearsonr
import random
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
import pingouin as pg
import seaborn as sns

## Read Processed Data

In [2]:
# load overall df
random_all_data = pd.read_csv("simulation_random/items_per_LLM_random_simulation.csv")
semi_random_all_data = pd.read_csv("simulation_semi_random/items_per_LLM_semi_random_simulation.csv")


## Normalize item scores where tasks have different scales on different items
- AUDIT, FTND, GABS, PG

In [3]:
def normalize_per_scale(data):
    scales_to_normalize = ["AUDIT scale", "FTND scale", "GABS scale", "PG scale"]

    data = data.copy()

    # Loop over each experiment you want to normalize
    for scale in scales_to_normalize:
        mask = data["experiment"] == scale

        if mask.sum() == 0:
            continue  # skip if scale not present

        # Normalize score
        data.loc[mask, "score"] = (
            data.loc[mask, "score"] - data.loc[mask, "score"].min()
        ) / (data.loc[mask, "score"].max() - data.loc[mask, "score"].min())

        # Normalize score_top_n
        data.loc[mask, "score_top_n"] = (
            data.loc[mask, "score_top_n"] - data.loc[mask, "score_top_n"].min()
        ) / (data.loc[mask, "score_top_n"].max() - data.loc[mask, "score_top_n"].min())

    return data

random_all_data = normalize_per_scale(random_all_data)
semi_random_all_data = normalize_per_scale(semi_random_all_data)


## Chronbach's Alpha

In [4]:
# function:
def compute_cronbach_alpha(data, score = "score"):
    results = []

    # loop through experiments
    for exp, exp_data in data.groupby("experiment"):
        
        # check whether this experiment has subcategories
        if exp_data["category"].notna().any() and exp != "SOEP scale":
            # compute alpha per category (domain)
            for domain, domain_data in exp_data.groupby("category"):
                df_wide = domain_data.pivot_table(
                    index="model", columns="item", values=score
                )
                #print(df_wide)
                if df_wide.shape[1] > 1:
                    alpha, ci = pg.cronbach_alpha(df_wide)
                else:
                    alpha, ci = None, (None, None)
                
                results.append({
                    "experiment": exp,
                    "domain": domain,
                    "alpha": alpha,
                    "alpha_CI": ci
                })
        else:
            # compute alpha for the whole experiment
            df_wide = exp_data.pivot_table(
                index="model", columns="item", values=score
            )
            #print(exp, df_wide.std().describe())
            if df_wide.shape[1] > 1:
                alpha, ci = pg.cronbach_alpha(df_wide)
            else:
                alpha, ci = None, (None, None)
            
            results.append({
                "experiment": exp,
                "domain": "total",  # no subdomain
                "alpha": alpha,
                "alpha_CI": ci
            })

    # convert to DataFrame
    return(pd.DataFrame(results))



In [5]:
# ---- Cronbach's alpha per domain or directly per scale, depending on whether the scale has subdomains ----
alpha_df_random = compute_cronbach_alpha(random_all_data)
alpha_df_semi_random = compute_cronbach_alpha(semi_random_all_data)


# ---- Cronbach's alpha per domain or directly per scale, depending on whether the scale has subdomains ----
alpha_df_random_top_n= compute_cronbach_alpha(random_all_data, score = "score_top_n")
alpha_df_semi_random_top_n = compute_cronbach_alpha(semi_random_all_data, score = "score_top_n")



# Merge only selected columns
alpha_df = alpha_df_random.merge(
    alpha_df_semi_random,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("_random", "_semi_random")
)

display(alpha_df)

,experiment,domain,alpha_random,alpha_CI_random,alpha_semi_random,alpha_CI_semi_random
0,AUDIT scale,total,-0.071954,"[-0.597, 0.335]",-0.141059,"[-0.717, 0.297]"
1,BARRAT scale,BISa,0.102869,"[-0.35, 0.447]",-0.164382,"[-0.752, 0.282]"
2,BARRAT scale,BISm,0.006240,"[-0.48, 0.383]",-0.085113,"[-0.616, 0.327]"
3,BARRAT scale,BISn,-0.253252,"[-0.867, 0.222]",0.044790,"[-0.423, 0.407]"
4,BART task,total,0.189144,"[-0.198, 0.498]",0.078136,"[-0.362, 0.429]"
5,CARE scale,CAREa,-0.024930,"[-0.536, 0.366]",-0.000305,"[-0.499, 0.382]"
6,CARE scale,CAREs,0.249995,"[-0.144, 0.542]",-0.329132,"[-1.027, 0.188]"
7,CARE scale,CAREw,-0.113835,"[-0.751, 0.332]",0.150673,"[-0.336, 0.491]"
8,CCT task,total,-0.350410,"[-1.039, 0.195]",-0.128017,"[-0.703, 0.327]"
9,DAST scale,total,0.119812,"[-0.296, 0.45]",0.177855,"[-0.211, 0.486]"


## Split-half Reliability

In [6]:
def split_half_reliability(df_items, n_splits=100):
    """Compute average split-half reliability (Spearman-Brown corrected)."""

    k = df_items.shape[1]
    
    if k < 2:
        return None, None

    cols = df_items.columns
    results = []

    for _ in range(n_splits):
        shuffled = np.random.permutation(cols)
        half1 = shuffled[:k//2]
        half2 = shuffled[k//2:]

        s1 = df_items[half1].sum(axis=1)
        s2 = df_items[half2].sum(axis=1)

        r, _ = pearsonr(s1, s2)

        if np.isfinite(r):
            r_sb = (2 * r) / (1 + r)  # Spearman-Brown correction
            results.append(r_sb)

    if len(results) == 0:
        return None, None
    
    return np.mean(results), np.std(results)


In [7]:
def compute_split_half_reliability(data, score="score"):
    rows = []  # final list of results

    for exp, exp_data in data.groupby("experiment"):

        # case 1: experiments with domains
        if exp_data["category"].notna().any() and exp != "SOEP scale":
            for domain, domain_data in exp_data.groupby("category"):

                df_wide = domain_data.pivot_table(
                    index="model", columns="item", values=score
                )

                mean_rsb, sd_rsb = split_half_reliability(df_wide)

                rows.append({
                    "experiment": exp,
                    "domain": domain,
                    "split_half_mean": mean_rsb,
                    "split_half_sd": sd_rsb
                })

        # case 2: experiments without domains
        else:
            df_wide = exp_data.pivot_table(
                index="model", columns="item", values=score
            )

            mean_rsb, sd_rsb = split_half_reliability(df_wide)

            rows.append({
                "experiment": exp,
                "domain": "total",
                "split_half_mean": mean_rsb,
                "split_half_sd": sd_rsb
            })

    return pd.DataFrame(rows)


In [8]:
split_half_reliability_random = compute_split_half_reliability(random_all_data)
split_half_reliability_semi_random = compute_split_half_reliability(semi_random_all_data)


# Merge 
split_half_rel_df = split_half_reliability_random.merge(
    split_half_reliability_semi_random,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("_random", "_semi_random")
)

display(split_half_rel_df)

,experiment,domain,split_half_mean_random,split_half_sd_random,split_half_mean_semi_random,split_half_sd_semi_random
0,AUDIT scale,total,-0.080495,0.200710,-0.159539,2.136689e-01
1,BARRAT scale,BISa,0.109307,0.174434,-0.146032,1.979904e-01
2,BARRAT scale,BISm,-0.036885,0.190173,-0.099002,2.161319e-01
3,BARRAT scale,BISn,-0.275524,0.207577,0.047008,2.043358e-01
4,BART task,total,0.194854,0.178293,0.064737,1.917134e-01
5,CARE scale,CAREa,-0.031379,0.203514,-0.068217,3.134879e-01
6,CARE scale,CAREs,0.396294,0.050320,-0.361338,3.090467e-01
7,CARE scale,CAREw,-0.166096,0.178635,0.160480,9.626586e-02
8,CCT task,total,-0.342467,0.286301,-0.150597,2.526352e-01
9,DAST scale,total,0.107847,0.140859,0.172181,1.711601e-01
